In [136]:
import pandas as pd

In [137]:
# Load the dataset
df = pd.read_csv('../01_Data/train.csv')

In [138]:
# Get columns with missing values and their null counts
null_counts = df.isnull().sum()
null_columns = null_counts[null_counts > 0]
print(null_columns)
# Iterate and print value counts along with null count
for col in null_columns.index:
    print(f"Column: {col} (Missing: {null_columns[col]})")
    print(df[col].value_counts(dropna=True))
    print("\n" + "-"*40 + "\n")


Age         177
Cabin       687
Embarked      2
dtype: int64
Column: Age (Missing: 177)
Age
24.00    30
22.00    27
18.00    26
28.00    25
30.00    25
         ..
24.50     1
0.67      1
0.42      1
34.50     1
74.00     1
Name: count, Length: 88, dtype: int64

----------------------------------------

Column: Cabin (Missing: 687)
Cabin
G6             4
C23 C25 C27    4
B96 B98        4
F2             3
D              3
              ..
E17            1
A24            1
C50            1
B42            1
C148           1
Name: count, Length: 147, dtype: int64

----------------------------------------

Column: Embarked (Missing: 2)
Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

----------------------------------------



In [139]:
# Fill missing Age with median
df['Age'] = df['Age'].fillna(df['Age'].median())

# Fill missing Embarked with the mode (most common)
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])

# Optionally drop Cabin (too many missing values)
df = df.drop(columns=['Cabin'])



In [140]:
df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [141]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['Sex'] = le.fit_transform(df['Sex'])  # female=0, male=1

In [142]:
df = pd.get_dummies(df, columns=['Embarked'], prefix='Embarked')


In [143]:
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,False,False,True
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,True,False,False
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,False,False,True
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,False,False,True
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,False,False,True


In [144]:
def transform_name(df):
    # Extract title using regex
    df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

    # Simplify rare titles
    df['Title'] = df['Title'].replace(
        ['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 
        'Sir', 'Jonkheer', 'Dona'], 'Rare')
    df['Title'] = df['Title'].replace({'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs'})


In [145]:
# # Extract title using regex
# df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

# # Simplify rare titles
# df['Title'] = df['Title'].replace(
#     ['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 
#      'Sir', 'Jonkheer', 'Dona'], 'Rare')
# df['Title'] = df['Title'].replace({'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs'})
transform_name(df)

In [146]:
df['Title'].value_counts()

Title
Mr        517
Miss      185
Mrs       126
Master     40
Rare       23
Name: count, dtype: int64

In [147]:
def tranform_age(df):
    # Create age bins (this leaves NaN unchanged)
    df['AgeGroup'] = pd.cut(df['Age'], bins=[0, 12, 20, 40, 60, 80], 
                            labels=['Child', 'Teen', 'Adult', 'Middle-Aged', 'Senior'])

In [148]:
tranform_age(df)
# Create age bins (this leaves NaN unchanged)
# df['AgeGroup'] = pd.cut(df['Age'], bins=[0, 12, 20, 40, 60, 80], 
#                         labels=['Child', 'Teen', 'Adult', 'Middle-Aged', 'Senior'])

In [149]:
df['AgeGroup'].value_counts()

AgeGroup
Adult          562
Middle-Aged    128
Teen           110
Child           69
Senior          22
Name: count, dtype: int64

In [150]:
df = pd.get_dummies(df, columns=['AgeGroup'], prefix='AgeGroup')
df = pd.get_dummies(df, columns=['Title'], prefix='Title')



In [151]:
print(df.columns.to_list())

['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'AgeGroup_Child', 'AgeGroup_Teen', 'AgeGroup_Adult', 'AgeGroup_Middle-Aged', 'AgeGroup_Senior', 'Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Rare']


In [152]:
df.drop(['Age', 'Name','Ticket','PassengerId'], axis=1, inplace=True)


In [153]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df['Fare'] = scaler.fit_transform(df[['Fare']])


In [154]:
df.to_csv("../03_Outputs/Preprocessed Train.csv",index=False)

In [155]:
#preprocessing of test dataset

# Load test data
df_test = pd.read_csv(r'../01_Data/test.csv')
df_test_label = pd.read_csv(r"..\01_Data\gender_submission.csv")
df_test = pd.merge(df_test, df_test_label, on='PassengerId', how='left')

In [156]:
df_test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
Survived         0
dtype: int64

In [157]:
# Fill missing Age with median
df_test['Age'] = df_test['Age'].fillna(df_test['Age'].median())
df_test['Fare'] = df_test['Fare'].fillna(df_test['Fare'].median())

In [158]:
df_test.isna().sum()
#cabin coilumn will be removed

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
Survived         0
dtype: int64

In [ ]:


# 1. Label encode 'Sex' using the trained encoder
df_test['Sex'] = le.transform(df_test['Sex'])

# 2. Extract 'Title' from 'Name' and simplify as in train
transform_name(df_test)
# df_test['Title'] = df_test['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
# df_test['Title'] = df_test['Title'].replace(
#     ['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 
#      'Sir', 'Jonkheer', 'Dona'], 'Rare')
# df_test['Title'] = df_test['Title'].replace({'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs'})

# 3. Create AgeGroup bins (NaNs will stay NaN)
tranform_age(df_test)
# df_test['AgeGroup'] = pd.cut(df_test['Age'], bins=[0, 12, 20, 40, 60, 80], 
#                              labels=['Child', 'Teen', 'Adult', 'Middle-Aged', 'Senior'])

# 4. One-hot encode 'Embarked', 'AgeGroup', and 'Title'
df_test = pd.get_dummies(df_test, columns=['Embarked', 'AgeGroup', 'Title'], prefix=['Embarked', 'AgeGroup', 'Title'])

# 5. Normalize 'Fare' using the scaler fitted on train
df_test['Fare'] = scaler.transform(df_test[['Fare']])

# 6. Drop unused columns like in train
df_test.drop(['Age', 'Name', 'PassengerId', 'Ticket'], axis=1, inplace=True)

# 7. Ensure test has the *same columns* as train
# For any missing columns in test that appeared in train, add them with zeros

missing_cols = set(df.columns) - set(df_test.columns) 
print(missing_cols)
#if missing dummies
for col in missing_cols:
    df_test[col] = 0


# Reorder columns to match train dataset exactly
df_test = df_test[df.columns]  # drop target column from train columns


set()


In [160]:
df_test.to_csv("../03_Outputs/Preprocessed Test.csv",index=False)